# Load libraries

In [ ]:
# import [package name] as [name abbr.]
# python 處理數字跟 tensor 運算的主要套件
import numpy as np
# google 的 NN coding 套件
import tensorflow as tf

# 有可能會出現警告，但是可以不用理他
# c:\users\silver\appdata\local\programs\python\python35\lib\site-packages\h5py\__init__.py:36:
# FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating`
# is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
# from ._conv import register_converters as _register_converters

import matplotlib.pyplot as plt

# Load Dataset

In [ ]:
exec(open("00_readingInput.py").read())
feature_dim = X_train.shape[1]
classes = 5

In [ ]:
# Define Model Input (x) and Output (y_),  y_ = f(x)
x = tf.placeholder(tf.float32, [None, X_train.shape[1]])
y_ = tf.placeholder(tf.int32, [None])
y_one = tf.one_hot(y_,classes)

#  Define Your Model Here

In [ ]:
#(Do!) define the model here



In [ ]:
# Define the Model Loss (4)
ce_loss = tf.reduce_mean( )
ce_train_step = tf.train.GradientDescentOptimizer(0.01).minimize(ce_loss)

# Accuracy of the Model
ce_y_pred = tf.argmax(tf.nn.softmax(ce_y), 1, output_type=tf.int32)
ce_correct = tf.equal(ce_y_pred, y_)
ce_accuracy = tf.reduce_mean(tf.cast(ce_correct, tf.float32))

#  Reference model (mse loss)

In [ ]:
mse_h1 = tf.layers.dense(x,128,activation=None)
mse_h1_act = tf.nn.sigmoid(mse_h1)
mse_h2 = tf.layers.dense(mse_h1_act,256,activation=None)
mse_h2_act = tf.nn.sigmoid(mse_h2)
mse_y = tf.layers.dense(mse_h2_act,classes,activation=None)
mse_y_softmax = tf.nn.softmax(mse_y)

In [ ]:
mse_loss = tf.reduce_mean(tf.losses.mean_squared_error(labels=y_one, predictions=mse_y_softmax))
mse_train_step = tf.train.GradientDescentOptimizer(0.01).minimize(mse_loss)

# Accuracy of the Model
mse_y_pred = tf.argmax(tf.nn.softmax(mse_y), 1, output_type=tf.int32)
mse_correct = tf.equal(mse_y_pred, y_)
mse_accuracy = tf.reduce_mean(tf.cast(mse_correct, tf.float32))

In [ ]:
# initialize the models
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

# Training & Evaluation

In [ ]:
epoch = 30
batch_size = 16
get_batches = gen_batches(X_train, Y_train, batch_size)
batches_in_a_epoch = int(X_train.shape[0]/batch_size)

In [ ]:
# Train Model
ce_hist_train_acc = []
ce_hist_valid_acc = []
mse_hist_train_acc = []
mse_hist_valid_acc = []
ce_hist_train_loss = []
ce_hist_valid_loss = []
mse_hist_train_loss = []
mse_hist_valid_loss = []

for step in range(epoch*batches_in_a_epoch):
    batch_xs, batch_ys = next(get_batches)
    sess.run(ce_train_step, feed_dict={x: batch_xs, y_: batch_ys})
    sess.run(mse_train_step, feed_dict={x: batch_xs, y_: batch_ys})
    if (step % batches_in_a_epoch == 0):
        # trainin and validation evaluation
        # cross entropy
        train_loss, train_acc = sess.run([ce_loss,ce_accuracy], feed_dict={x: X_train, y_: Y_train})
        valid_loss, valid_acc = sess.run([ce_loss,ce_accuracy], feed_dict={x: X_test,  y_: Y_test})
        ce_hist_train_acc.append(train_acc)
        ce_hist_valid_acc.append(valid_acc)
        ce_hist_train_loss.append(train_loss)
        ce_hist_valid_loss.append(valid_loss)
        #print("[CE]  Accuracy: [T] %.4f / [V] %.4f" % (train_acc,valid_acc))
        # mse
        train_loss, train_acc = sess.run([mse_loss,mse_accuracy], feed_dict={x: X_train, y_: Y_train})
        valid_loss, valid_acc = sess.run([mse_loss,mse_accuracy], feed_dict={x: X_test,  y_: Y_test})
        mse_hist_train_acc.append(train_acc)
        mse_hist_valid_acc.append(valid_acc)
        mse_hist_train_loss.append(train_loss)
        mse_hist_valid_loss.append(valid_loss)
        #print("[MSE] Accuracy: [T] %.4f / [V] %.4f" % (train_acc,valid_acc))
sess.close()

In [ ]:
plt.figure(0,figsize=(9,6))
plt.subplot(121)
x = [x for x in range(len(ce_hist_train_loss))]
ce_line, = plt.plot(x, ce_hist_train_loss, label='CE')
mse_line, = plt.plot(x, mse_hist_train_loss, label='MSE')
plt.xlabel('#epoch')
plt.ylabel('Loss')
plt.ylim([0,2])
plt.legend(handles=[ce_line,mse_line], loc=4)
plt.subplot(122)
x = [x for x in range(len(ce_hist_train_acc))]
ce_line, = plt.plot(x, ce_hist_train_acc, label='CE')
mse_line, = plt.plot(x, mse_hist_train_acc, label='MSE')
plt.xlabel('#epoch')
plt.ylabel('Accuracy')
plt.legend(handles=[ce_line,mse_line], loc=4)
plt.savefig('./training_curve/ex01_LossFunction.png')
plt.show()
plt.close()